wine 품질(quality) 예측

In [2]:
import numpy as np
import pandas as pd
import plotly.express as px

df_white = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases\/wine-quality/winequality-white.csv', sep=';')
df_red = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases\/wine-quality/winequality-red.csv', sep=';')
df_wine_predic = pd.concat([df_white, df_red])

# px.histogram(df_wine_predic.quality)
# df_wine_predic.head()

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns

# ax = sns.heatmap(df_wine_predic.corr(), annot=True, annot_kws={'size':6})
# plt.show()

In [3]:
from sklearn.model_selection import train_test_split

df_wine_predic.columns = df_wine_predic.columns.str.replace(' ', '_')
train, test = train_test_split(df_wine_predic, test_size=0.3, random_state=0,stratify=df_wine_predic['quality'])

train_X = train[train.columns[:11]]
train_Y = train[train.columns[11:]]
test_X = test[test.columns[:11]]
test_Y = test[test.columns[11:]]

train_Y

,quality
1685,7
366,6
3826,6
3441,5
1903,5
...,...
4308,6
2650,5
4512,6
4607,6


In [4]:
########### 인공 신경망 외에 다른 모델로 예측 모델링 진행해 보세요.

In [5]:
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier

def Mining_algo(model, name):
    model.fit(train_X, np.ravel(train_Y))
    pred = model.predict(test_X)
    print(f'{name} 예측 정확도:', metrics.accuracy_score(pred, test_Y))


Mining_algo(DecisionTreeClassifier(), "디시전 트리")
Mining_algo(RandomForestClassifier(), "랜덤 포레스트")
Mining_algo(AdaBoostClassifier(), "아다부스트")
Mining_algo(KNeighborsClassifier(), "KNN")

디시전 트리 예측 정확도: 0.56
랜덤 포레스트 예측 정확도: 0.6641025641025641
아다부스트 예측 정확도: 0.4666666666666667
KNN 예측 정확도: 0.4723076923076923


############ 인공 신경망<br><br>
데이터 준비: 와인 데이터셋을 불러옴. 전처리 수행 <br>
모델 정의: 다중 클래스 분류를 위한 인공 신경망 모델 정의<br>
손실 함수와 최적화 알고리즘 선택: 다중 클래스 분류에 적합한 손실 함수와 최적화 알고리즘 선택 <br>
모델 학습: 학습 데이터를 사용하여 모델 학습 <br>
모델 평가: 테스트 데이터를 사용하여 모델 평가 <br>

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset

# 데이터 준비
X = df_wine_predic.drop('quality', axis=1).values
y = df_wine_predic['quality'].values

# 학습 데이터와 테스트 데이터 분리
train_X, test_X, train_y, test_y = train_test_split(X,y,
                                                    test_size=0.2,
                                                    stratify=y)


# 데이터 표준화
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(train_X)
X_test_scaled = scaler.transform(test_X)

In [7]:
# PyTorch 텐서로 변환
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(train_y, dtype=torch.long)
y_test_tensor = torch.tensor(test_y, dtype=torch.long)

# DataLoader 생성
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# 배치 16 설정
train_loader = DataLoader(dataset=train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=16, shuffle=False)

In [8]:
print(train_X.shape)
print(train_y.min(),train_y.max())

(5197, 11)
3 9


In [84]:
# 모델 정의
class WineQualityNet(nn.Module):
    def __init__(self):
        super(WineQualityNet,self).__init__()
        self.fc1 = nn.Linear(11, 16)
        self.hidden1 = nn.Linear(16,8)
        self.fc2 = nn.Linear(8, 10)
        self.dropout = nn.Dropout(p=0.2)
        self.relu = nn.ReLU()
        self.rrelu = nn.RReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.rrelu(self.fc1(x))
        x = self.dropout(x)
        x = self.rrelu(self.hidden1(x))
        x = self.fc2(x)

        return x
    
# 정답 코드
# 모델 정의
# class WineQualityNet(nn.Module):
#     def __init__(self):
#         super(WineQualityNet,self).__init__()
#         self.fc1 = nn.Linear(11, 128)
#         self.fc2 = nn.Linear(128, 64)
#         self.fc3 = nn.Linear(64, 10)  # 0부터 9까지의 품질 값을 예측하기 위해 출력층의 크기는 10
#         self.relu = nn.ReLU()

#     def forward(self, x):
#         x = self.relu(self.fc1(x))
#         x = self.relu(self.fc2(x))
#         x = self.fc3(x)
#         return x

#         return x
# # 모델 인스턴스 생성
# model_wind = WineQualityNet()

# # 손실 함수와 최적화 알고리즘 선택
# crit = nn.CrossEntropyLoss()
# opti = optim.Adam(model_wind.parameters(), lr = 0.002)

# #가중치 편향
# print(model_wind.fc1.bias.data.shape, model_wind.fc2.weight.data.shape)
# 모델 인스턴스 생성
model_wind = WineQualityNet()

# 손실 함수와 최적화 알고리즘 선택
crit = nn.CrossEntropyLoss()
opti = optim.Adam(model_wind.parameters(), lr = 0.002)

#가중치 편향
print(model_wind.fc1.bias.data.shape, model_wind.fc2.weight.data.shape)

torch.Size([128]) torch.Size([64, 128])


In [87]:
# 모델 학습
for epoch in range(300):
    total_loss = 0.0
    for inputs, labels in train_loader:
        opti.zero_grad()  # 기울기 초기화 0으로
        outputs = model_wind(inputs)  # x를 넣어 순전파 과정을 통해 최종 값 아웃풋
        losses = crit(outputs, labels)  # CrossEntropy 손실 계산
        losses.backward()  # 역전파
        opti.step()  # 매개 변수를 갱신하며 업데이트
        total_loss += losses.item()  # 현재 배치의 손실을 더함.

    # 평균 손실 출력 (10번의 에포크마다 한 번씩)
    if (epoch+1) % 10 == 0:
        print(f'Epoch {epoch+1}, Loss: {total_loss/len(train_loader)}')

Epoch 10, Loss: 0.3027684921599351
Epoch 20, Loss: 0.28877253437271483
Epoch 30, Loss: 0.24254030723984424
Epoch 40, Loss: 0.22268041046766135
Epoch 50, Loss: 0.24103090085089207
Epoch 60, Loss: 0.20628967928771788
Epoch 70, Loss: 0.19634010408646785
Epoch 80, Loss: 0.17203848123550414
Epoch 90, Loss: 0.15489896183117077
Epoch 100, Loss: 0.20443035438083684
Epoch 110, Loss: 0.2041103749894179
Epoch 120, Loss: 0.18415017328869837
Epoch 130, Loss: 0.14604125622803202
Epoch 140, Loss: 0.1266566950863657
Epoch 150, Loss: 0.17325322977911967
Epoch 160, Loss: 0.10864084921633968
Epoch 170, Loss: 0.15564494088721964
Epoch 180, Loss: 0.15039676671847702
Epoch 190, Loss: 0.11960347819142043
Epoch 200, Loss: 0.18356723084042853
Epoch 210, Loss: 0.11915243885694789
Epoch 220, Loss: 0.15019702999709317
Epoch 230, Loss: 0.06202653533886545
Epoch 240, Loss: 0.1734597732016566
Epoch 250, Loss: 0.08713697933126241
Epoch 260, Loss: 0.11702243205553924
Epoch 270, Loss: 0.07407105584019938
Epoch 280, Los

In [88]:
# 수정된 모델 평가 진행
model_wind.eval() 
with torch.no_grad(): 
    total = 0 
    correct = 0
    for inputs, labels in test_loader:
        outputs = model_wind(inputs) 
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the test set: %d %%' % (100 * correct / total))

Accuracy of the network on the test set: 64 %
